In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
g_places_api_key = env.g_places_api_key

In [37]:
import json

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.reviews'  # Specify the fields you want
}

# Define the query
data = {
    'textQuery': 'YUMMY JUICE BAR'
}


In [38]:

# Make the POST request
response = requests.post(url, headers=headers, json=data)

# Print the response
if response.ok:
    places = response.json()['places']
    for place in places:
        print(json.dumps(place, indent=2))
else:
    print(f'Error: {response.status_code}')
    print(response.text)


{
  "id": "ChIJ58zUFZT1wokR6iBMBLDp5OA",
  "formattedAddress": "737 Lydig Ave, Bronx, NY 10462, USA",
  "displayName": {
    "text": "Yummy Juice Bar",
    "languageCode": "en"
  },
  "reviews": [
    {
      "name": "places/ChIJ58zUFZT1wokR6iBMBLDp5OA/reviews/ChdDSUhNMG9nS0VJQ0FnSURwdFBxejdRRRAB",
      "relativePublishTimeDescription": "2 months ago",
      "rating": 5,
      "text": {
        "text": "Love the Acai Berry bowl. I customize my order; can't do bananas so I replace with pineapples.\nAs much as I love it, I can only do one a week. The calorie count is pretty high. The acai berry is 97 calories (per serving as per the sales lady) not to mention the fruits, coconut shavings and the Nutella (love, love, love it). So yeap!, once a week treat.",
        "languageCode": "en"
      },
      "originalText": {
        "text": "Love the Acai Berry bowl. I customize my order; can't do bananas so I replace with pineapples.\nAs much as I love it, I can only do one a week. The calorie

In [ ]:

"name": "places/ChIJ58zUFZT1wokR6iBMBLDp5OA/reviews/ChdDSUhNMG9nS0VJQ0FnSURwdFBxejdRRRAB",
"relativePublishTimeDescription": "2 months ago",
"rating": 5,
"text": {
"text": "Love the Acai Berry bowl. I customize my order; can't do bananas so I replace with pineapples.\nAs much as I love it, I can only do one a week. The calorie count is pretty high. The acai berry is 97 calories (per serving as per the sales lady) not to mention the fruits, coconut shavings and the Nutella (love, love, love it). So yeap!, once a week treat.",
"languageCode": "en"
},
"originalText": {
"text": "Love the Acai Berry bowl. I customize my order; can't do bananas so I replace with pineapples.\nAs much as I love it, I can only do one a week. The calorie count is pretty high. The acai berry is 97 calories (per serving as per the sales lady) not to mention the fruits, coconut shavings and the Nutella (love, love, love it). So yeap!, once a week treat.",
"languageCode": "en"
},
"authorAttribution": {
"displayName": "BEE R.O.D.",
"uri": "https://www.google.com/maps/contrib/118334706736474348616/reviews",
"photoUri": "https://lh3.googleusercontent.com/a-/ALV-UjXLGg_2ENuV_Sk8bSrG_hj2liS84-SJeo4JXGKt_GtiTXWx=s128-c0x00000000-cc-rp-mo"
},
"publishTime": "2023-08-17T00:27:09Z"


In [14]:
# Assuming place_id is the Place ID of the place you're interested in
place_id = 'ChIJ58zUFZT1wokR6iBMBLDp5OA'

url = f'https://places.googleapis.com/v1/{place_id}'#/reviews:search'

# Headers remain the same
# ...

# Make the POST request
response = requests.post(url, headers=headers)

# Print the response
if response.ok:
    reviews = response.json()['reviews']
    for review in reviews:
        print(json.dumps(review, indent=2))
else:
    print(f'Error: {response.status_code}')
    print(response.text)


Error: 404



In [30]:
# Replace with the actual Place ID of the place you're interested in
place_id = 'ChIJ58zUFZT1wokR6iBMBLDp5OA'

url = f'https://places.googleapis.com/v1/places/{place_id}?fields=id,displayName,formattedAddress,reviews&key={g_places_api_key}'

# Make the GET request
response = requests.get(url)

# Print the response
if response.ok:
    place_data = response.json()
    print(json.dumps(place_data, indent=2))
else:
    print(f'Error: {response.status_code}')
    print(response.text)

{
  "id": "ChIJ58zUFZT1wokR6iBMBLDp5OA",
  "formattedAddress": "737 Lydig Ave, Bronx, NY 10462, USA",
  "displayName": {
    "text": "Yummy Juice Bar",
    "languageCode": "en"
  },
  "reviews": [
    {
      "name": "places/ChIJ58zUFZT1wokR6iBMBLDp5OA/reviews/ChdDSUhNMG9nS0VJQ0FnSURwdFBxejdRRRAB",
      "relativePublishTimeDescription": "2 months ago",
      "rating": 5,
      "text": {
        "text": "Love the Acai Berry bowl. I customize my order; can't do bananas so I replace with pineapples.\nAs much as I love it, I can only do one a week. The calorie count is pretty high. The acai berry is 97 calories (per serving as per the sales lady) not to mention the fruits, coconut shavings and the Nutella (love, love, love it). So yeap!, once a week treat.",
        "languageCode": "en"
      },
      "originalText": {
        "text": "Love the Acai Berry bowl. I customize my order; can't do bananas so I replace with pineapples.\nAs much as I love it, I can only do one a week. The calorie

In [28]:
# Make the POST request
response = requests.post(url, headers=headers)

# Print the response
if response.ok:
    reviews = response.json()['reviews']
    for review in reviews:
        print(json.dumps(review, indent=2))
else:
    print(f'Error: {response.status_code}')
    print(response.text)


Error: 404



In [26]:
response

<Response [404]>